In [ ]:
import numpy as np
import tensorflow as tf
 #1.14.0import nodepy.linear_multistep_method as lm
import timeit

np.random.seed(1234)
tf.random.set_seed(1234)

class Multistep_NN:
    def __init__(self, dt, X, layers, M, scheme):

        self.dt = dt
        self.X = X # S x N x D
        
        self.S = X.shape[0] # number of trajectories
        self.N = X.shape[1] # number of time snapshots
        self.D = X.shape[2] # number of dimensions
        
        self.M = M # number of Adams-Moulton steps
        
        self.layers = layers
        
        # Load weights
        switch = {'AM': lm.Adams_Moulton,
                  'AB': lm.Adams_Bashforth,
                  'BDF': lm.backward_difference_formula}
        method = switch[scheme](M)
        self.alpha = np.float32(-method.alpha[::-1])
        self.beta = np.float32(method.beta[::-1])
                
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
        
        self.X_tf = tf.placeholder(tf.float32, shape=[self.S, None, self.D]) # S x N x D
        self.X_star_tf = tf.placeholder(tf.float32, shape=[None, self.D]) # N_star x D
        
        scope_name = str(np.random.randint(1e6))
        with tf.variable_scope(scope_name) as scope:
            self.f_pred = self.neural_net(self.X_star_tf) # N_star x D
        with tf.variable_scope(scope, reuse=True):
            self.Y_pred = self.net_Y(self.X_tf) # S x N x D
        
        self.loss = self.D*tf.reduce_mean(tf.square(self.Y_pred))
        
        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)
                
    def neural_net(self, H):
        num_layers = len(self.layers)
        for l in range(0,num_layers-2):
            with tf.variable_scope("layer%d" %(l+1)):
                H = tf.layers.dense(inputs=H, units=self.layers[l+1], activation=tf.nn.tanh)
        with tf.variable_scope("layer%d" %(num_layers-1)):
            H = tf.layers.dense(inputs=H, units=self.layers[-1], activation=None)
        return H
    
    def net_F(self, X): # S x (N-M+1) x D
        X_reshaped = tf.reshape(X, [-1,self.D]) # S(N-M+1) x D
        F_reshaped = self.neural_net(X_reshaped) # S(N-M+1) x D
        F = tf.reshape(F_reshaped, [self.S,-1,self.D]) # S x (N-M+1) x D
        return F # S x (N-M+1) x D
    
    def net_Y(self, X): # S x N x D
        M = self.M
        
        Y = self.alpha[0]*X[:,M:,:] + self.dt*self.beta[0]*self.net_F(X[:,M:,:])
        for m in range(1, M+1):
            Y = Y + self.alpha[m]*X[:,M-m:-m,:] + self.dt*self.beta[m]*self.net_F(X[:,M-m:-m,:]) # S x (N-M+1) x D
        
        return Y # S x (N-M+1) x D
    
    def train(self, N_Iter):
        
        tf_dict = {self.X_tf: self.X}
        
        start_time = timeit.default_timer()
        for it in range(N_Iter):
            
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print
            if it % 10 == 0:
                elapsed = timeit.default_timer() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                print('It: %d, Loss: %.3e, Time: %.2f' % 
                      (it, loss_value, elapsed))
                start_time = timeit.default_timer()

    
    def predict_f(self, X_star):
        
        F_star = self.sess.run(self.f_pred, {self.X_star_tf: X_star})
        
        return F_star

In [ ]:

import numpy as np
import matplotlib as mpl
#mpl.use('pgf')

def figsize(scale, nplots = 1):
    fig_width_pt = 390.0                          # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = nplots*fig_width*golden_mean              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots to inherit fonts from the document
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 10,
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.figsize": figsize(1.0),     # default fig size of 0.9 textwidth
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",    # use utf8 fonts becasue your computer can handle it :)
        r"\usepackage[T1]{fontenc}",        # plots will be generated using this preamble
        ]
    }
mpl.rcParams.update(pgf_with_latex)

import matplotlib.pyplot as plt

# I make my own newfig and savefig functions
def newfig(width, nplots = 1):
    fig = plt.figure(figsize=figsize(width, nplots))
    ax = fig.add_subplot(111)
    return fig, ax

def savefig(filename, crop = True):
    if crop == True:
#        plt.savefig('{}.pgf'.format(filename), bbox_inches='tight', pad_inches=0)
        plt.savefig('{}.pdf'.format(filename), bbox_inches='tight', pad_inches=0)
        plt.savefig('{}.eps'.format(filename), bbox_inches='tight', pad_inches=0)
    else:
#        plt.savefig('{}.pgf'.format(filename))
        plt.savefig('{}.pdf'.format(filename))
        plt.savefig('{}.eps'.format(filename))


In [9]:


import numpy as np
from scipy.integrate import odeint

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.gridspec as gridspec

def colorline3d(ax, x, y, z, cmap):
    N = len(x)
    skip = int(0.01*N)
    for i in range(0,N,skip):
        ax.plot(x[i:i+skip+1], y[i:i+skip+1], z[i:i+skip+1], color=cmap(int(255*i/N)))

if __name__ == "__main__": 
    
    # function that returns dx/dt
    def f(x,t): # x is 3 x 1
        sigma = 10.0
        beta = 8.0/3.0
        rho = 28.0
        
        f1 = sigma*(x[1]-x[0])
        f2 = x[0]*(rho-x[2])-x[1]
        f3 = x[0]*x[1]-beta*x[2]
        f = np.array([f1,f2,f3])
        return f
        
    # time points
    t_star = np.arange(0,25,0.01)
    
    # initial condition
    x0 = np.array([-8.0, 7.0, 27])
    
    # solve ODE
    X_star = odeint(f, x0, t_star)
    
    noise = 0.00
    
    skip = 1
    dt = t_star[skip] - t_star[0]
    X_train = X_star[0::skip,:]
    X_train = X_train + noise*X_train.std(0)*np.random.randn(X_train.shape[0], X_train.shape[1])
    
    X_train = np.reshape(X_train, (1,X_train.shape[0],X_train.shape[1]))

    layers = [3, 256, 3]
    
    M = 1
    scheme = 'AM'
    model = Multistep_NN(dt, X_train, layers, M, scheme)
    
    N_Iter = 50000
    model.train(N_Iter)
    
    def learned_f(x,t):
        f = model.predict_f(x[None,:])
        return f.flatten()
    
    learned_X_star = odeint(learned_f, x0, t_star)
    
    ####### Plotting ################## 
    fig, ax = newfig(1.0, 0.8)
    ax.axis('off')
    
    gs0 = gridspec.GridSpec(1, 2)
    gs0.update(top=0.95, bottom=0.1, left=0.0, right=0.90, wspace=0.15)
    
    ax = plt.subplot(gs0[:, 0:1], projection='3d')
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    colorline3d(ax, X_star[:,0], X_star[:,1], X_star[:,2], cmap = plt.cm.ocean)
    ax.grid(False)
    ax.set_xlim([-20,20])
    ax.set_ylim([-50,50])
    ax.set_zlim([0,50])
    ax.set_xticks([-20,0,20])
    ax.set_yticks([-40,0,40])
    ax.set_zticks([0,25,50])
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.set_title('Exact Dynamics', fontsize = 10)
    
    ax = plt.subplot(gs0[:, 1:2], projection='3d')
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))    
    colorline3d(ax, learned_X_star[:,0], learned_X_star[:,1], learned_X_star[:,2], cmap = plt.cm.ocean)
    ax.grid(False)
    ax.set_xlim([-20,20])
    ax.set_ylim([-50,50])
    ax.set_zlim([0,50])
    ax.set_xticks([-20,0,20])
    ax.set_yticks([-40,0,40])
    ax.set_zticks([0,25,50])
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.set_title('Learned Dynamics', fontsize = 10)
    
    # savefig('./figures/Lorenz', crop = False)
    
    ####### Plotting ##################
    
    fig, ax = newfig(1.0, 1.5)
    ax.axis('off')
    
    gs0 = gridspec.GridSpec(3, 1)
    gs0.update(top=0.95, bottom=0.15, left=0.1, right=0.95, hspace=0.5)
    
    ax = plt.subplot(gs0[0:1, 0:1])
    ax.plot(t_star,X_star[:,0],'r-')
    ax.plot(t_star,learned_X_star[:,0],'k--')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$x$')
    
    ax = plt.subplot(gs0[1:2, 0:1])
    ax.plot(t_star,X_star[:,1],'r-')
    ax.plot(t_star,learned_X_star[:,1],'k--')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$y$')
    
    ax = plt.subplot(gs0[2:3, 0:1])
    ax.plot(t_star,X_star[:,2],'r-',label='Exact Dynamics')
    ax.plot(t_star,learned_X_star[:,2],'k--',label='Learned Dynamics')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$z$')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=2, frameon=False)


AttributeError: module 'tensorflow' has no attribute 'Session'